In [1]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
# imports
import tempfile

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI
from selenium import webdriver
import time
import requests
from urllib.parse import quote
import os
from pydub import AudioSegment
from pydub.playback import play
from io import BytesIO
import ffmpeg
import tempfile
import subprocess
from io import BytesIO
from pydub import AudioSegment
import time


In [2]:
# Initialization

load_dotenv()

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
MODEL = "gpt-4o-mini"
openai = OpenAI()

OpenAI API Key exists and begins sk-proj-


In [3]:
system_message = (
    "You are an assistant designed exclusively for generating NFL game recaps. "
    "All outputs must be related to NFL games and align with the given input. "
    "You require a week, team, and a musical artist to generate a recap. "
    "Without a specified musical artist, inform the user to provide one before proceeding. "
    "The generated content must reflect the musical style of the provided artist, "
    "while maintaining the integrity of the NFL game information. "
    "Your output should transform the game recap into a inspired by the artist's style."
    "You must provide exactly 2 verses and 2 choruses"
    "If the user mentions something that is not about football, redirect the conversation back to your task.You can engage the user's initial text briefly and nicely, but the focus is football"
    
)


In [4]:
def get_games(week):
    url=f'https://www.pff.com/nfl/scores?week={week}'
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)  # Adjust sleep time as needed for content to load
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    main_div = soup.find('main', id='main')
    links=["https://www.pff.com"+link.get('href')+'#recap' for link in main_div.find_all('a')]
    driver.quit()
    return links
    

In [5]:
def identify_team(links,team):
    system_message = (
    "You are an assistant capable of finding URLs related to NFL teams."
    " Your response must ONLY return the URL as a plain string, without any additional text or explanation."
    )
    user_prompt = (
    f"From the list of links {links}, identify the one containing the team '{team}', "
    "and return ONLY the link as a plain string. For example: "
    "'https://www.pff.com/nfl/scores/2024/1/carolina-panthers_at_new-orleans-saints_25900#recap'."
    )
    prompts = [
    {"role": "system", "content": system_message},
    {"role": "user", "content": user_prompt}
    ]

    completion = openai.chat.completions.create(
    model='gpt-4o-mini',
    messages=prompts,
    )
    
    return completion.choices[0].message.content

In [6]:
def get_content(url):
    driver = webdriver.Chrome()
    driver.get(url)
    time.sleep(5)  # Adjust sleep time as needed for content to load
    
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    main_div = soup.find('main', id='main')
    if main_div:
        content_div=main_div.find_all('div',class_='m-content')[0].find_all('div',class_='m-longform-copy')[0]
        if content_div:
            content=content_div.get_text(separator=" ",strip=True)
            print('Content Gathered Successfully')
        else:
            print("m-content div not found within main div")
    else:
        print("Main div not found")
    
    driver.quit()
    return content

In [7]:
def validate_and_generate(week, team, artist, url):
    content = get_content(url)  # Fetch recap content
    if content:
        system_message = (
            "You are an assistant designed exclusively for generating NFL game recaps. "
            "Your task is to transform the provided recap content into a song inspired by the style of a specified musical artist. "
            "The song must have 2 verses and 2 choruses"
            "All outputs must be based on the content provided below and formatted in the artist's style, "
            "while remaining focused on the NFL game details. "
            "Here is the recap content: "
            f"'{content}'."
        )
        user_prompt = (
            f"Generate a {artist} style recap for the {team} in week {week}. "
            "Use the provided content and keep it fresh."
        )
    else:
        system_message = (
            "You are an assistant that generates NFL recaps. "
            "There is no recap content available for week {week} for the {team}. "
            "Inform the user that the week has no recap yet."
        )
        user_prompt = (
            f"Sorry, week {week} has no recap for the {team}. Let the user know."
        )

    prompts = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt},
    ]

    completion = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=prompts,
    )
    return completion.choices[0].message.content

In [8]:
week_function={
        "name": "get_games",
        "description": "Get NFL game information for a specific week.",
        "parameters": {
            "type": "object",
            "properties": {
                "week": {
                    "type": "string",
                    "description": "The NFL week number (e.g., '1', '2', '3')."
                },
                "team": {
                    "type": "string",
                    "description": "The team to get the game information for."
                },
                "style": {
                    "type": "string",
                    "description": "The artist style to use for the game recap."
                },
            },
            "required": ["week"],
        },
    }

In [9]:
tools = [ {"type":"function","function":week_function}]

In [10]:
def chat(message, history):
    """
    Handles chatbot responses and streams audio output for client playback.
    """
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason == "tool_calls":
        message = response.choices[0].message
        response = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        response = openai.chat.completions.create(model=MODEL, messages=messages)

        response = openai.chat.completions.create(model=MODEL, messages=messages)
        messages.append({'role': 'assistant', 'content': response.choices[0].message.content})

    # Extract the assistant's reply
    reply = response.choices[0].message.content

    # Generate audio for the reply
    response_audio = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",
        input=reply
    )
    audio_stream = BytesIO(response_audio.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")

    # Play locally
    #play_audio(audio)

    # Save the audio to a temporary file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio_file:
        audio.export(temp_audio_file.name, format="mp3")
        temp_audio_path = temp_audio_file.name

    return reply, temp_audio_path  # Return text reply and file path

In [11]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    print('Tool Triggered')
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)  # Ensure it's JSON
    week = arguments.get('week')
    team = arguments.get('team')
    artist = arguments.get('artist')
    links = get_games(week)
    team_game= identify_team(links,team)    
    recap=validate_and_generate(week, team, artist, team_game)
    response= {
        "role": "tool",
        "content": recap,
        "tool_call_id": message.tool_calls[0].id
    }
    return response

In [12]:
def play_audio(audio_segment):
    temp_dir = tempfile.gettempdir()
    temp_path = os.path.join(temp_dir, "temp_audio.wav")
    print(f"Temporary file path: {temp_path}")
    print(os.path.exists(temp_path))
    try:
        audio_segment.export(temp_path, format="wav")
        time.sleep(3) # Student Dominic found that this was needed. You could also try commenting out to see if not needed on your PC
        subprocess.call([
            "ffplay",
            "-nodisp",
            "-autoexit",
            "-hide_banner",
            temp_path
        ], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    finally:
        try:
            os.remove(temp_path)
        except Exception:
            pass
 
def talker(message):
    response = openai.audio.speech.create(
        model="tts-1",
        voice="onyx",  # Also, try replacing onyx with alloy
        input=message
    )
    audio_stream = BytesIO(response.content)
    audio = AudioSegment.from_file(audio_stream, format="mp3")
    play_audio(audio)

In [ ]:
import gradio as gr

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(label="Chatbot", type="messages")
    audio_output = gr.Audio(label="Assistant's Reply", type="filepath", autoplay=True)

    state = gr.State({"history": []})

    def process_message(message, state):
        response_text, response_audio = chat(message, state["history"])
        state["history"].append({"role": "user", "content": message})
        state["history"].append({"role": "assistant", "content": response_text})
        return state["history"], response_audio

    input_box = gr.Textbox(label="Your Message", placeholder="Type a message...")
    send_button = gr.Button("Send")

    send_button.click(
        process_message,
        inputs=[input_box, state],
        outputs=[chatbot, audio_output]
    )

demo.launch(share=True,debug=True)


* Running on local URL:  http://127.0.0.1:7875
* Running on public URL: https://4ea15125486f1d8a51.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Tool Triggered
Content Gathered Successfully
Tool Triggered
Main div not found


Traceback (most recent call last):
  File "C:\Users\bunka\anaconda\Lib\site-packages\gradio\queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bunka\anaconda\Lib\site-packages\gradio\route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bunka\anaconda\Lib\site-packages\gradio\blocks.py", line 2015, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bunka\anaconda\Lib\site-packages\gradio\blocks.py", line 1562, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\bunka\anaconda\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return await get_async_backend().run_sync_in_worker_thread(
      

Tool Triggered
Content Gathered Successfully
Tool Triggered
Content Gathered Successfully
Tool Triggered
Content Gathered Successfully
Tool Triggered
Content Gathered Successfully
